In [1]:
import numpy as np, pandas as pd, scanpy as sc, matplotlib.pyplot as plt, os
from scipy.stats import hypergeom
import celloracle as co, glob, pickle
from functools import reduce
from tqdm import tqdm
import itertools, math, random
import networkx as nx
from itertools import combinations
from scipy.stats import gmean

# visualization settings required to see plots in jupyter notebook
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.rcParams['figure.figsize'] = [6, 4.5]
plt.rcParams["savefig.dpi"] = 300

wd = '/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/cicero_outs'
out_path = os.path.join(wd, 'out_data')
os.makedirs(f"{out_path}/figures", exist_ok=True)
os.makedirs(f"{out_path}/out_files", exist_ok=True)
sc.settings.figdir = f"{out_path}/figures"
random.seed(42)

In [2]:
!which bedtools

/opt/packages/bedtools/2.30.0/bedtools


In [3]:
from celloracle import motif_analysis as ma

ref_genome = "hg38"
genomes_dir = '/ocean/projects/cis240075p/skeshari/utils_data/genomes/'
genome_installation = ma.is_genome_installed(ref_genome=ref_genome, genomes_dir=genomes_dir)
print(ref_genome, "installation: ", genome_installation)
if not genome_installation:
    import genomepy
    genomepy.install_genome(name=ref_genome, provider="UCSC", genomes_dir=genomes_dir)
else:
    print(ref_genome, "is installed already.")

peaks = pd.read_csv(f"/ocean/projects/cis240075p/skeshari/igvf/bcell2/male_donor/out_data/cicero_output/all_peaks.csv", header=0, index_col=0)
peaks = peaks.x.values
cicero_connections = pd.read_csv(f"/ocean/projects/cis240075p/skeshari/igvf/bcell2/male_donor/out_data/cicero_output/cicero_connections.csv", header=0, index_col=0).dropna()
tss_annotated = ma.get_tss_info(peak_str_list=peaks, ref_genome="hg38")
integrated = ma.integrate_tss_peak_with_cicero(tss_peak=tss_annotated, cicero_connections=cicero_connections)

hg38 installation:  True
hg38 is installed already.


***** WARNING: File /tmp/pybedtools.8xhe57lh.tmp has inconsistent naming convention for record:
KI270728.1	232189	233167

***** WARNING: File /tmp/pybedtools.8xhe57lh.tmp has inconsistent naming convention for record:
KI270728.1	232189	233167



que bed peaks: 191255
tss peaks in que: 23030
